In [1]:
import geopandas
import pandas as pd
import numpy as np
from shapely.geometry import Point
import time
import random

## Readin Data

In [2]:
county_shp = geopandas.GeoDataFrame.from_file('county/县级行政区.shp')
county_shp['ID'] = [i for i in range(county_shp.shape[0])]
county_shp.head()

,PAC,NAME,省代码,省,市代码,市,类型,geometry,ID
0,110101,东城区,110000,北京市,110000,北京市,市辖区,"POLYGON ((116.40581 39.96245, 116.40783 39.960...",0
1,110102,西城区,110000,北京市,110000,北京市,市辖区,"POLYGON ((116.38139 39.96006, 116.38053 39.956...",1
2,110105,朝阳区,110000,北京市,110000,北京市,市辖区,"MULTIPOLYGON (((116.48030 40.07965, 116.48970 ...",2
3,110106,丰台区,110000,北京市,110000,北京市,市辖区,"POLYGON ((116.31980 39.89578, 116.31978 39.894...",3
4,110107,石景山区,110000,北京市,110000,北京市,市辖区,"POLYGON ((116.14485 39.99233, 116.14568 39.991...",4


In [3]:
prov_shp = geopandas.GeoDataFrame.from_file('prov/省级行政区.shp')
prov_shp.head()

,省代码,省,类型,geometry
0,110000,北京市,直辖市,"POLYGON ((116.67527 41.04010, 116.67616 41.040..."
1,120000,天津市,直辖市,"POLYGON ((117.44383 40.25101, 117.45611 40.246..."
2,130000,河北省,省,"MULTIPOLYGON (((118.85390 39.10692, 118.84934 ..."
3,140000,山西省,省,"POLYGON ((114.13714 40.73445, 114.13860 40.732..."
4,150000,内蒙古自治区,自治区,"POLYGON ((121.49813 53.32607, 121.50116 53.321..."


In [4]:
# prov_id -> ID
prov_id2ID = {}
for prov_id in prov_shp['省代码']:
    prov_id2ID[prov_id] = county_shp[county_shp['省代码'] == prov_id].ID.tolist()

In [5]:
# 县
geom = county_shp.geometry
county_centroids = geom.centroid  # 质心
county_total_bounds = geom.total_bounds  # 总边界
# 省
prov_geom = prov_shp.geometry

/home/u2017201647/.conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


## Function Definition

In [ ]:
# 是否处于四方边界内
def within_bound(lo, lat, bound):
    if lo < bound[0] or lo > bound[2] or lat < bound[1] or lat > bound[3]:
        return False
    else:
        return True
    
# 处理坐标字符串
# [lo, lat, name]
def readin_center(filename, year_str):
    center_coords = []
    
    for line in open(filename, 'r').readlines():
        cur_center_coord = []
        
        coords = line.replace('-'+year_str+'.png', '').split('-')
        cur_center_coord.append(float(coords[1]))  # lo
        cur_center_coord.append(float(coords[0]))  # lat
        cur_center_coord.append(line)  # name
        
        center_coords.append(cur_center_coord)
        
    return center_coords

In [ ]:
for year in ['2016', '2017', '2018', '2019', '2020']:
    print('==========' + year + '==========')
    txt_name = 'Data/center/' + year+'_pictures.txt'
    
    center_coor = readin_center(txt_name, year)
    center_coor = pd.DataFrame(center_coor).values
    
    start = time.time()
    match_lst = []
    match_PAC_lst = []
    for i, center in enumerate(center_coor):
        if (i+1)%10000 == 0:
            print('===', i+1, '===')
            print('accumulated time:', time.time()-start)
        
        # 当前点
        lo = center[0]; lat = center[1]; name = center[2]
        point = Point(lo, lat)
        
        # 不在整体范围内
        if not within_bound(lo, lat, county_total_bounds):
            # print('Out of bounds!')
            match_lst.append(-1)
            match_PAC_lst.append(None)
            continue
        
        # 搜索匹配省份
        prov_ids = prov_shp[prov_geom.intersects(point)]['省代码'].tolist()
        match = False
        # 匹配省份
        for prov_id in prov_ids:
            ID_lst = prov_id2ID[prov_id]
            # 省份内县级单元
            for idx in ID_lst:
                poly = geom[idx]
                if poly.intersects(point):
                    match = True
                    match_lst.append(idx)
                    match_PAC_lst.append(county_shp.PAC[idx])
                    break
                
            if match:
                break
            
        if not match:
            match_lst.append(-1)
            match_PAC_lst.append(None)
            
    
    match = pd.DataFrame({'lo': center_coor[:, 0],
                          'lat': center_coor[:, 1],
                          'name': center_coor[:, 2],
                          'PAC': match_PAC_lst})
    match.to_csv('Data/center2PAC'+year+'.csv', index = False)